## Comparing a signal with itself while adding noise

With this notebook you can compare an audio signal with itself while adding noise to it using the AudioFP class for fingerprinting. The idea is to get a sense of how noise affects audio fingerprinting and the comparison of fingerprints.

In [ ]:
# Import relevant packages

from bokeh.io import output_notebook
import warnings
import sys
import AudioFP as afp
import os

warnings.filterwarnings('ignore')
output_notebook()

### Fingerprint a song

We start by first fingerprinting a song. 

In [ ]:
# Create AudioFP object for first song 
song1 = afp.AudioFP(process='a')  # When prompted to choose whether to read audiofile or saved fingerprint, 
                                  # enter "f" to read from audiofile
                                  # because we need the raw signal to add noise.

### Add noise to the signal and fingerprint the noisy track

Next, we create another `AudioFP` object. This time we proceed manually by setting the `process` argument to `m`. We start by creating an empty object and manually setting its `framerate` and `songname` properties based on the original signal. Next, we will use the function `add_noise` defined in the `AudioFP` class to generate Gaussian white noise of a specified decibel level and add to the signal. See [this page](https://chchearing.org/noise/common-environmental-noise-levels/) for common noise levels in decibels. The function `add_noise` takes the audio signal and its framerate as inputs in that order and outputs the signal with the added noise. Finally, we will go through the steps to generate a fingerprint of the signal with the noise.

In [ ]:
# Create another AudioFP object from the same file and add noise
song2 = afp.AudioFP(process='m')
plot = False  # boolean to display results
song2.songname = 'noisy_' + song1.songname 
filename = song1.songname
channels, song2.framerate = afp.AudioFP.read_audiofile(song2, plot, filename)
# Add noise to the signal
channels = afp.add_noise(channels, song2.framerate)
# Create audio fingerprint
f, t, sgram = afp.AudioFP.generate_spectrogram(song2, plot, channels, song2.framerate)
fp, tp, peaks = afp.AudioFP.find_peaks(song2, plot, f, t, sgram)
afp.AudioFP.generate_fingerprint(song2, plot, fp, tp, peaks)

### Comparing fingerprints

For comparing two fingerprints, we will calculate what is known as the Jaccard similarity. Jaccard similarity, mathematically is the size of the intersection divided by the size of the union between two givent sets. Thus, two identical sets would have a Jaccard similarity index of 1 while entirely dissimilar sets would result in 0. A number in between 0 and 1 indicates some similarity, however, there isn't any rule specifying how "similar" are two songs with a Jaccard similarity index of say 0.7 for instance. All we can say at this point is that closer the Jaccard similarity index of two songs is to 1 the more similar they are. One could use [bootstrapping](https://en.wikipedia.org/wiki/Bootstrapping_(statistics)) to determine the extent of similarity of an arbitrary similarity score. Below, we have used some ranges based on some intuition using a small set of songs. The function `compare_fingerprints` is defined in the `AudioFP` class. If you want to see how the ranges are defined, take a look at the file `AudioFP.py`.

In [ ]:
afp.compare_fingerprints(song1, song2)